In [1]:
# SPARK SESSIONS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import col, dayofmonth, month, year, date_format
import findspark
import zipfile
import os
from pyspark.sql.functions import when, lit, col, current_timestamp, input_file_name
import pandas as pd
import requests
import tarfile
import requests
import json
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession

/tmp/ipykernel_18640/1330914320.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# import os
# import findspark
# from pyspark.sql import SparkSession
print("SPARK_HOME:", os.environ.get('SPARK_HOME'))
print("JAVA_HOME:", os.environ.get('JAVA_HOME'))

findspark.init()
spark = SparkSession.builder.appName("sparksql").getOrCreate()


SPARK_HOME: /opt/spark
JAVA_HOME: /usr/lib/jvm/java-17-openjdk-amd64


your 131072x1 screen size is bogus. expect trouble
25/11/09 14:51:22 WARN Utils: Your hostname, r resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/09 14:51:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 14:51:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
try:
    spark.stop()
except:
    pass

In [ ]:
# %load_ext sparksql_magic
# %config SparkSql.max_num_rows=20

In [6]:
%load_ext sparksql_magic
%config SparkSql.max_num_rows=1000

/home/r/git_projekty/sparkTraining/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: UserWarning: Config option `max_num_rows` not recognized by `SparkSql`.
  if await self.run_code(code, result, async_=asy):


In [5]:
# general paths
DATA = '/home/r/git_projekty/sparkTraining/data'
PACKAGED_DATA = '/home/r/git_projekty/sparkTraining/data/packaged'
UNPACKAGED_DATA = '/home/r/git_projekty/sparkTraining/data/unpackaged'

In [5]:
# file1 = '/home/r/pyspark/sparkTraining/New_Query_2025-08-01_12_01pm_2025_08_08.csv'
foler_zip = '/home/r/pgetl/sparkTraining/data.tar.gz'

In [5]:
def extract_zip_file(zip_path: str, extract_to: str):
    try:
        if not os.path.exists(extract_to):
            os.makedirs(extract_to)
            print(f"Created directory: {extract_to}")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"Extracted all files to: {extract_to}")
    except FileNotFoundError as e:
        print(f"Error: The file {zip_path} was not found. {e}")
    except zipfile.BadZipFile as e:
        print(f"Error: The file {zip_path} is not a zip file or it is corrupted. {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [46]:
packaged_data_path = PACKAGED_DATA
print( packaged_data_path)
files_to_unpack = os.listdir(packaged_data_path) #'csecicids2018.zip'
data = '/home/r/git_projekty/sparkTraining/data'
for file_to_unpack in files_to_unpack:
    # print(file_to_unpack,'++')
    if file_to_unpack.endswith('.zip'):
            extract_to = file_to_unpack.replace('.zip', '')
            # print( extract_to)
    if extract_to.startswith('.'):
        # print( extract_to, '========')
        extract_to = extract_to.replace('.', '')
    #     print( extract_to, '--')
    # print( extract_to, '<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    extract_zip_file(os.path.join(packaged_data_path, file_to_unpack), os.path.join(UNPACKAGED_DATA, extract_to))
# extract_to = file_to_unpack.replace('.zip', '')
# extract_zip_file(os.path.join(packaged_data_path, file_to_unpack), os.path.join(packaged_data_path, extract_to))

/home/r/git_projekty/sparkTraining/data/packaged
Created directory: /home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets
Extracted all files to: /home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets
Created directory: /home/r/git_projekty/sparkTraining/data/unpackaged/csecicids2018
Extracted all files to: /home/r/git_projekty/sparkTraining/data/unpackaged/csecicids2018
Created directory: /home/r/git_projekty/sparkTraining/data/unpackaged/game-recommendations-on-steam
Extracted all files to: /home/r/git_projekty/sparkTraining/data/unpackaged/game-recommendations-on-steam
Created directory: /home/r/git_projekty/sparkTraining/data/unpackaged/us-wildfire-records-6th-edition
Extracted all files to: /home/r/git_projekty/sparkTraining/data/unpackaged/us-wildfire-records-6th-edition


/home/r/git_projekty/sparkTraining/data/packaged/csesisids2018.zip
False


In [6]:
def unziptar(folder_zip, output_folder):
    with tarfile.open(folder_zip, 'r:gz') as tar:
        tar.extractall(path=output_folder)

    print(f"Plik {folder_zip} został rozpakowany do katalogu {output_folder}")

In [ ]:
# output_folder = '/home/r/pgetl/sparkTraining/extracted_data'

In [34]:
# unziptar(foler_zip, output_folder)

In [4]:
# 1. Definicja ścieżki do pliku
csv_file_path = "/home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets/transactions_data.csv"

def read_csv_with_spark(spark: SparkSession, file_path: str) -> DataFrame:
    """
    Wczytuje plik CSV do DataFrame Spark z określonymi opcjami.

    :param spark: SparkSession
    :param file_path: Ścieżka do pliku CSV
    :return: DataFrame Spark
    """
    df = spark.read.csv(
        file_path,
        header=True,       # Użyj pierwszej linii jako nagłówków kolumn
        inferSchema=False,  # Spróbuj automatycznie wykryć typy danych (np. Int, String, Date)
        sep=",",           # Określ separator (domyślnie przecinek, ale można zmienić na ';', '\t' itd.)
        encoding="UTF-8"   # Określ kodowanie pliku
    )
    return df

df_csv = read_csv_with_spark(spark, csv_file_path)

# 3. Wyświetlenie pierwszych wierszy i schematu
print("--- Pierwsze 5 wierszy ---")
df_csv.show(5)

print("--- Schemat (struktura danych) ---")
df_csv.printSchema()

--- Pierwsze 5 wierszy ---
+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|         use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00|Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57|Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00|Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00|Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00

In [5]:
df_csv.createOrReplaceTempView("transactionsdata")

In [6]:
%%sparksql
select * from transactionsdata limit 2

id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,null
7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,null


In [3]:
# 1. Definicja ścieżki do pliku
json_file_path = "/home/r/git_projekty/sparkTraining/data/unpackaged/game-recommendations-on-steam/games_metadata.json"

def read_json_with_spark(spark: SparkSession, file_path: str) -> DataFrame:
    """
    Wczytuje plik CSV do DataFrame Spark z określonymi opcjami.

    :param spark: SparkSession
    :param file_path: Ścieżka do pliku CSV
    :return: DataFrame Spark
    """
    df = spark.read.json(
        file_path,
        multiLine=False,    # Wczytaj JSON wieloliniowy, jeśli plik jest w takim formacie
        encoding="UTF-8"   # Określ kodowanie pliku
    )
    return df

df_json = read_json_with_spark(spark, json_file_path)

# 3. Wyświetlenie pierwszych wierszy i schematu
print("--- Pierwsze 5 wierszy ---")
df_json.show(5)

print("--- Schemat (struktura danych) ---")
df_json.printSchema()

--- Pierwsze 5 wierszy ---
+------+--------------------+--------------------+
|app_id|         description|                tags|
+------+--------------------+--------------------+
| 13500|Enter the dark un...|[Action, Adventur...|
| 22364|                    |            [Action]|
|113020|Monaco: What's Yo...|[Co-op, Stealth, ...|
|226560|Escape Dead Islan...|[Zombies, Adventu...|
|249050|Dungeon of the En...|[Roguelike, Strat...|
+------+--------------------+--------------------+
only showing top 5 rows

--- Schemat (struktura danych) ---
root
 |-- app_id: long (nullable = true)
 |-- description: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [4]:
df_json.createOrReplaceTempView("trainfraudlabels")

In [7]:
%%sparksql
select * from trainfraudlabels limit 2

app_id,description,tags
13500,"Enter the dark underworld of Prince of Persia Warrior Within, the sword-slashing sequel to the critically acclaimed Prince of Persia: The Sands of Time™. Hunted by Dahaka, an immortal incarnation of Fate seeking divine retribution, the Prince embarks upon a path of both carnage and mystery to defy his preordained death.","['Action', 'Adventure', 'Parkour', 'Third Person', 'Great Soundtrack', 'Singleplayer', 'Platformer', 'Time Travel', 'Atmospheric', 'Classic', 'Hack and Slash', 'Time Manipulation', 'Gore', 'Fantasy', 'Story Rich', 'Dark', 'Open World', 'Controller', 'Dark Fantasy', 'Puzzle']"
22364,,['Action']


In [6]:
def load_json_files_to_df(spark: SparkSession, file_path: str) -> DataFrame:
    """
    Ładuje pliki JSON z podanego folderu do DataFrame, używając istniejącej sesji Spark.
    Args:
        spark: Aktywna sesja Spark.
        folder_path: Ścieżka do folderu zawierającego pliki JSON.
    Returns:
        DataFrame PySpark z załadowanymi danymi lub pusty DataFrame, jeśli nie znaleziono pasujących plików.
    """
    try:
        # Wczytanie wszystkich plików JSON z folderu
        # df = spark.read.json(f"{folder_path}/*.json")
        df = spark.read.json(f"{file_path}")
        return df
    except Exception as e:
        print(f"Wystąpił błąd podczas ładowania plików JSON: {e}")
        # Zwrócenie pustego DataFrame w przypadku błędu
        return spark.createDataFrame([], StructType([]))

In [7]:
folder_path = 'transactions-fraud-datasets'
file_json = 'train_fraud_labels.json'
print( os.path.join(UNPACKAGED_DATA, folder_path, file_json))
df = load_json_files_to_df(spark, os.path.join(UNPACKAGED_DATA, folder_path, file_json))

/home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets/train_fraud_labels.json


25/11/09 14:18:57 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 8]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeUnalignedBytes(UnsafeWriter.java:127)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:110)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.datasources.FileFormat$$anon$1.$anonfun$apply$1(FileFormat.scala:156)
	at org.apache.spark.sql.execution.datasources.FileFormat$$anon$1$$Lambda$3019/0x000070c6ece7f588.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.next(FileScanRDD.

Wystąpił błąd podczas ładowania plików JSON: py4j does not exist in the JVM


ConnectionRefusedError: [Errno 111] Connection refused

In [55]:
# Przykład użycia z już otwartą sesją Spark
# Zakładając, że masz już aktywną sesję Spark o nazwie 'spark'

folder_path = 'transactions-fraud-datasets'
dataframes = {}
files_json = [f for f in os.listdir(os.path.join(UNPACKAGED_DATA, folder_path)) if f.endswith('.json')]
print( files_json)
for file_json in files_json:
    full_path = os.path.join(UNPACKAGED_DATA, folder_path, file_json)
    print( full_path)
    df = load_json_files_to_df(spark, os.path.join(UNPACKAGED_DATA, folder_path, file_json))
    dataframes[file_json] = df
# my_dataframe = load_json_files_to_df(spark, os.join(UNPACKAGED_DATA, folder_path))

# if my_dataframe.count() > 0:
#     my_dataframe.printSchema()
#     my_dataframe.show()

# # Utworzenie tymczasowego widoku o nazwie 'my_temp_view'
# my_dataframe.createOrReplaceTempView("my_temp_view")

['train_fraud_labels.json', 'mcc_codes.json']
/home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets/train_fraud_labels.json


25/11/09 12:10:22 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 9)/ 8]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.getBinary(UnsafeRow.java:393)
	at org.apache.spark.sql.catalyst.json.CreateJacksonParser$.internalRow(CreateJacksonParser.scala:84)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$.$anonfun$inferFromDataset$4(JsonDataSource.scala:106)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$$$Lambda/0x00007ad62cf9f9b0.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.$anonfun$infer$3(JsonInferSchema.scala:86)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$Lambda/0x00007ad62d0259c8.apply(Unknown Source)
	at org.apache.spark.util.SparkErrorUtils.tryWithResource(SparkErrorUtils.scala:47)
	at org.apache.spark.util.SparkErrorUtils.tryWithResource$(SparkErrorUtils.scala:46)
	at org.apache.spark.util.Utils$.tryWithResource(Utils.scal

Wystąpił błąd podczas ładowania plików JSON: py4j does not exist in the JVM


ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# 1. Definicja ścieżki do pliku
csv_file_path = "/home/r/git_projekty/sparkTraining/data/unpackaged/transactions-fraud-datasets/train_fraud_labels.json"

# 2. Wczytanie pliku CSV
df = spark.read.csv(
    csv_file_path,
    header=True,       # Użyj pierwszej linii jako nagłówków kolumn
    inferSchema=False,  # Spróbuj automatycznie wykryć typy danych (np. Int, String, Date)
    sep=",",           # Określ separator (domyślnie przecinek, ale można zmienić na ';', '\t' itd.)
    encoding="UTF-8"   # Określ kodowanie pliku
)

# 3. Wyświetlenie pierwszych wierszy i schematu
print("--- Pierwsze 5 wierszy ---")
df.show(5)

print("--- Schemat (struktura danych) ---")
df.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused

In [9]:
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession

def load_json_files_to_df_(spark: SparkSession, folder_path: str, name_patterns: list) -> DataFrame:
    """
    Ładuje pliki JSON z podanego folderu do DataFrame, używając istniejącej sesji Spark.
    Filtruje pliki, które mają w nazwie wszystkie podane wzorce.

    Args:
        spark: Aktywna sesja Spark.
        folder_path: Ścieżka do folderu zawierającego pliki JSON.
        name_patterns: Lista stringów, które muszą znaleźć się w nazwie pliku.

    Returns:
        DataFrame PySpark z załadowanymi danymi lub pusty DataFrame, jeśli nie znaleziono pasujących plików.
    """

    # Tworzenie wzorca glob do odczytu plików
    file_pattern = folder_path + "/*" + "*".join(name_patterns) + "*.json"

    try:
        # Odczyt plików pasujących do wzorca
        df = spark.read.option("multiLine", "true").json(file_pattern)

        # Sprawdzenie, czy DataFrame jest pusty
        if df.rdd.isEmpty():
            print(f"Brak plików pasujących do wzorca '{file_pattern}'. Zwracam pusty DataFrame.")
            return spark.createDataFrame([], "struct<col:string>")

        # # Dodanie nowej kolumny 'file_name' z pełną ścieżką pliku
        # df_with_filename = df.withColumn("file_name", input_file_name())

        # Dodanie nowej kolumny 'file_name' z samą nazwą pliku
        # regexp_extract(input_file_name(), '([^/]+)$', 1)
        # ^ - początek stringa
        # [^/] - dowolny znak oprócz ukośnika
        # + - jeden lub więcej znaków
        # $ - koniec stringa
        df_with_filename = df.withColumn(
            "file_name",
            regexp_extract(input_file_name(), '([^/]+)$', 1)
        )
        print(f"Pomyślnie załadowano pliki do DataFrame.")
        return df_with_filename

    except Exception as e:
        print(f"Wystąpił błąd podczas ładowania plików: {e}")
        return spark.createDataFrame([], "struct<col:string>")

# Przykład użycia z już otwartą sesją Spark
# Zakładając, że masz już aktywną sesję Spark o nazwie 'spark'
# folder_path = "/path/to/your/json/folder"
# patterns = ['product_status_campaign_id', 'line_item_id']
# my_dataframe = load_json_files_to_df(spark, folder_path, patterns)

# if my_dataframe.count() > 0:
#     my_dataframe.printSchema()
#     my_dataframe.show()

In [10]:
# Przykład użycia z już otwartą sesją Spark
# Zakładając, że masz już aktywną sesję Spark o nazwie 'spark'
folder_path = "/home/r/pgetl/sparkTraining/extracted_data"
patterns = ['entity=product_status__campaign_id=', '__line_item_id=']
# entity=product_status__campaign_id=*__line_item_id=*__page=
my_dataframe = load_json_files_to_df(spark, folder_path, patterns)

if my_dataframe.count() > 0:
    my_dataframe.printSchema()
    my_dataframe.show()

# Utworzenie tymczasowego widoku o nazwie 'my_temp_view'
my_dataframe.createOrReplaceTempView("my_temp_view")

Wystąpił błąd podczas ładowania plików: [PATH_NOT_FOUND] Path does not exist: file:/home/r/pgetl/sparkTraining/extracted_data/*entity=product_status__campaign_id=*__line_item_id=*.json.


In [11]:
%%sparksql
SELECT * FROM my_temp_view order by api_product_id --limit 100

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `api_product_id` cannot be resolved. Did you mean one of the following? [`col`].; line 1 pos 36;
'Sort ['api_product_id ASC NULLS FIRST], true
+- Project [col#0]
   +- SubqueryAlias my_temp_view
      +- View (`my_temp_view`, [col#0])
         +- LogicalRDD [col#0], false


In [ ]:
# request po product status dla tej kampani dla tej adtroupy i zobaczyc co dostaniemy
# URL, z którym będziesz się łączyć
API_URL = "https://jsonplaceholder.typicode.com/posts"

# Zapytanie GET: Pobieranie danych z API
def request_get():
    print("--- Zapytanie GET ---")
    try:
        response = requests.get(API_URL)

        # Sprawdzenie, czy zapytanie zakończyło się sukcesem (kod 200)
        response.raise_for_status()

        # Pobranie danych jako JSON
        posts = response.json()
        print("Pobrano 100 postów.")

        # Wyświetlenie pierwszego posta
        print("Pierwszy post:")
        print(json.dumps(posts[0], indent=2))
        return posts
    except requests.exceptions.RequestException as e:
        print(f"Wystąpił błąd GET: {e}")

# ---

# Zapytanie POST: Wysyłanie nowych danych do API
def request_post():
    print("\n--- Zapytanie POST ---")
    new_post = {
        "title": "Tytuł nowego posta",
        "body": "Treść nowego posta",
        "userId": 1
    }

    try:
        post_response = requests.post(API_URL, json=new_post)
        post_response.raise_for_status()

        # Otrzymanie odpowiedzi od serwera
        created_post = post_response.json()
        print("Nowy post został utworzony:")
        print(json.dumps(created_post, indent=2))
        return created_post

    except requests.exceptions.RequestException as e:
        print(f"Wystąpił błąd POST: {e}")

# ---

# Zapytanie PUT: Aktualizowanie istniejących danych
def request_put():
    print("\n--- Zapytanie PUT ---")
    update_url = f"{API_URL}/1" # Aktualizujemy post o ID 1
    updated_data = {
        "id": 1,
        "title": "Zaktualizowany tytuł",
        "body": "Zaktualizowana treść",
        "userId": 1
    }

    try:
        put_response = requests.put(update_url, json=updated_data)
        put_response.raise_for_status()

        updated_post = put_response.json()
        print("Post został zaktualizowany:")
        print(json.dumps(updated_post, indent=2))
        return updated_post

    except requests.exceptions.RequestException as e:
        print(f"Wystąpił błąd PUT: {e}")

# ---

# Zapytanie DELETE: Usuwanie danych
# def request_delete():
#     print("\n--- Zapytanie DELETE ---")
#     delete_url = f"{API_URL}/1" # Usuwamy post o ID 1

#     try:
#         delete_response = requests.delete(delete_url)
#         delete_response.raise_for_status()

#         print("Post o ID 1 został usunięty.")

#     except requests.exceptions.RequestException as e:
#         print(f"Wystąpił błąd DELETE: {e}")

In [ ]:
print( df_with_filename.head() )

In [10]:
data_file1 = (
        spark.read.format("csv")
        .option("inferSchema", "true")
        .option("header", "true")
        .option("sep", ",")
        .load(file1)
    )

In [11]:
data_file1.createOrReplaceTempView("data_file1")

In [15]:
data_file1.write.format("parquet").save("data_file1.parquet")

In [ ]:
%%sparksql
SELECT * FROM data_file1 limit 2


only showing top 20 row(s)


retailer,sub_retailer,country,region,postal_code,scrape_time,keyword,ad_type,device,brand_raw,rank,product_title_id,product_id,price_amount,rating,review,product_url,image_url,oos,date,time,data_source,grid,paid,filename,product_title,price_unit,feature_label,store_id,requestor_id,month,year,source_file_name,source_last_mod_timestamp,dcom_load_timestamp,category,dcom_processing_timestamp
Walmart,NA,US,US,95829,2025-08-08 11:59:16.066616,aussie shampoo and conditioner,sp,mobile-app,null,1,NA,505105842,15.0,4.7,NA,https://www.walmart.com/ip/Aussie-Miracle-Moist-Shampoo-and-Conditioner-Hair-Set-26-2-fl-oz/505105842?classType=REGULAR&athbdg=L1200,https://i5.walmartimages.com/seo/Aussie-Miracle-Moist-Shampoo-and-Conditioner-Hair-Set-26-2-fl-oz_d37f65b2-7e60-41ea-a137-560d3db675bc.566f38993f9a9f17807a5987b8dcdeb6.jpeg?odnHeight=1500&odnWidth=1500&odnBg=ffffff,null,2025-08-08,11:59:16 AM,xbyte,True,True,ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,Aussie Miracle Moist Shampoo and Conditioner Dual Pack Hair Set 262 fl oz,USD,null,none,scheduler_on_VM,8,2025,abfss://raw@sa02flexflowdcom01prod.dfs.core.windows.net/schedule/xbyte/walmart/scheduler_on_VM/3eec335c-11e6-4497-8fbe-754d0f84d148/keyword_search/2025/08/ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,2025-08-08 11:59:18,2025-08-08 12:07:47.817000,null,2025-08-08 17:36:25.721487
Walmart,NA,US,US,95829,2025-08-08 11:59:16.066666,aussie shampoo and conditioner,sp,mobile-app,null,2,NA,1480516348,15.0,4.6,NA,https://www.walmart.com/ip/Aussie-Miracle-Curls-Shampoo-and-Conditioner-Dual-Pack-For-All-Hair-Types-26-2-oz/1480516348?classType=REGULAR&athbdg=L1102,https://i5.walmartimages.com/seo/Aussie-Miracle-Curls-Shampoo-and-Conditioner-Dual-Pack-For-All-Hair-Types-26-2-oz_68afe0a7-520d-4089-887c-3abccf5ddea6.2981a321dff289c2c9fef81e1ebf01d4.jpeg?odnHeight=1500&odnWidth=1500&odnBg=ffffff,null,2025-08-08,11:59:16 AM,xbyte,True,True,ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,Aussie Miracle Curls Shampoo and Conditioner Dual Pack For All Hair Types 262 oz,USD,null,none,scheduler_on_VM,8,2025,abfss://raw@sa02flexflowdcom01prod.dfs.core.windows.net/schedule/xbyte/walmart/scheduler_on_VM/3eec335c-11e6-4497-8fbe-754d0f84d148/keyword_search/2025/08/ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,2025-08-08 11:59:18,2025-08-08 12:07:47.817000,null,2025-08-08 17:36:25.721487
Walmart,NA,US,US,95829,2025-08-08 11:59:16.066689,aussie shampoo and conditioner,organic,mobile-app,null,3,NA,505105842,15.0,4.7,NA,https://www.walmart.com/ip/Aussie-Miracle-Moist-Shampoo-and-Conditioner-Hair-Set-26-2-fl-oz/505105842?classType=REGULAR&athbdg=L1200,https://i5.walmartimages.com/seo/Aussie-Miracle-Moist-Shampoo-and-Conditioner-Hair-Set-26-2-fl-oz_d37f65b2-7e60-41ea-a137-560d3db675bc.566f38993f9a9f17807a5987b8dcdeb6.jpeg?odnHeight=1500&odnWidth=1500&odnBg=ffffff,null,2025-08-08,11:59:16 AM,xbyte,True,False,ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,Aussie Miracle Moist Shampoo and Conditioner Dual Pack Hair Set 262 fl oz,USD,null,none,scheduler_on_VM,8,2025,abfss://raw@sa02flexflowdcom01prod.dfs.core.windows.net/schedule/xbyte/walmart/scheduler_on_VM/3eec335c-11e6-4497-8fbe-754d0f84d148/keyword_search/2025/08/ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_11-59-17.json,2025-08-08 11:59:18,2025-08-08 12:07:47.817000,null,2025-08-08 17:36:25.721487
Walmart,NA,US,US,95829,2025-08-08 11:59:16.066706,aussie shampoo and conditioner,organic,mobile-app,null,4,NA,1480516348,15.0,4.6,NA,https://www.walmart.com/ip/Aussie-Miracle-Curls-Shampoo-and-Conditioner-Dual-Pack-For-All-Hair-Types-26-2-oz/1480516348?classType=REGULAR&athbdg=L1102,https://i5.walmartimages.com/seo/Aussie-Miracle-Curls-Shampoo-and-Conditioner-Dual-Pack-For-All-Hair-Types-26-2-oz_68afe0a7-520d-4089-887c-3abccf5ddea6.2981a321dff289c2c9fef81e1ebf01d4.jpeg?odnHeight=1500&odnWidth=1500&odnBg=ffffff,null,2025-08-08,11:59:16 AM,xbyte,True,False,ab219ef7-b37a-470b-ba1a-db8666d221a7_20250808_1

In [ ]:
# kursy_walut_xml = "https://static.nbp.pl/dane/kursy/xml/a045z240304.xml"  # https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2024.csv
# kursy_walut_csv = {
#     "2020": "https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2020.csv",
#     "2021": "https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2021.csv",
#     "2022": "https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2022.csv",
#     "2023": "https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2023.csv",
#     "2024": "https://static.nbp.pl/dane/kursy/Archiwum/archiwum_tab_a_2024.csv",
# }

In [ ]:
def nic():
    # url = kursy_walut_csv["2024"]
    # s = requests.get(url).content
    # # pd_kursy = pd.read_csv(url, encoding='cp1250'   )
    # kursy = pd.read_csv(url, sep=";", encoding="cp1250", index_col="data")
    # pd_kursy = pd.read_csv(
    #     kursy_walut_csv["2024"], sep=";", encoding="cp1250", index_col="data"
    # )
    # unzip_files = [
    #     "2015.zip",
    #     "2016.zip",
    #     "2017.zip",
    #     "2019.zip",
    #     "detroit_911_calls.zip",
    #     "malaysia_covid_cases.zip",
    # ]
    # state_path = 'DATA/MATURY'
    # unzip_files2 = [ item for item in os.listdir(state_path) if item.endswith('.zip')]
    # print( unzip_files2 )
    # #################
    # def unzip_file(file_to_unzip):
    #     print(file_to_unzip.split("."))
    #     with zipfile.ZipFile(file_to_unzip, "r") as file:
    #         file.extractall(file_to_unzip.split(".")[0])
    # ###################
    # # for item in unzip_files2:
    # #     item = os.path.join(state_path, item )
    # #     print( item )
    # #     unzip_file( item )

    # ###################
    # files_to_proceed = ['marki.txt','perfumy.txt','sklad.txt']
    # paths_to_proceed = ['DATA/MATURY/informatyka-2019-maj-matura-rozszerzona-zalaczniki/Dane_PR']
    # ###################
    # marki =  os.path.join(paths_to_proceed[0], files_to_proceed[0])
    # perfumy =  os.path.join(paths_to_proceed[0], files_to_proceed[1])
    # sklad =  os.path.join(paths_to_proceed[0], files_to_proceed[2])
    # print( marki )
    # print( perfumy )
    # print( sklad )
    # data_marki = (
    #     spark.read.format("csv")
    #     .option("inferSchema", "true")
    #     .option("header", "true")
    #     .option("sep", "\t")
    #     .load(marki)
    # )
    # data_perfumy = (
    #     spark.read.format("csv")
    #     .option("inferSchema", "true")
    #     .option("header", "true")
    #     .option("sep", "\t")
    #     .load(perfumy)
    # )
    # data_sklad = (
    #     spark.read.format("csv")
    #     .option("inferSchema", "true")
    #     .option("header", "true")
    #     .option("sep", "\t")
    #     .load(sklad)
    # )
    # # data_marki = (
    # #     spark.read.text(file1)
    # # )
    # ###################
    # data_marki.show(5)
    # ###################
    # data_marki.createOrReplaceTempView("marki")
    # data_perfumy.createOrReplaceTempView("perfumy")
    # data_sklad.createOrReplaceTempView("sklad")
    # spark.sql("select * from marki").show(5)
    # spark.sql("select * from perfumy").show(5)
    # spark.sql("select * from sklad").show(5)
    # ###################
    # #Podaj listę wszystkich nazw perfum, których jednym ze składników jest „absolut jasminu”.
    # spark.sql('select * from perfumy where id_perfum in  (select id_perfum from sklad where nazwa_skladnika = "absolut jasminu")').show()
    # ###################
    # # Podaj  listę  różnych  rodzin  zapachów.  Dla  każdej  rodziny  podaj  jej  nazwę,  cenę  najtańszych
    # # perfum z tej rodziny i ich nazwę.  Więcej arkuszy znajdziesz na stronie: arkusze.pl
    # spark.sql('select rodzina_zapachow , count(*) from perfumy group by rodzina_zapachow' ).show()
    # spark.sql('select rodzina_zapachow , min(cena) as minimum from perfumy group by rodzina_zapachow order by minimum ' ).show()
    # ###################
    # marki = (
    #     spark
    #     .table("marki")
    # )
    # perfumy = (
    #     spark
    #     .table("perfumy")
    # )
    # sklad = (
    #     spark
    #     .table("sklad")
    # )
    # marki.show(5)
    # perfumy.show(5)
    # sklad.show(5)
    # sklad_absolut = (
    #     spark
    #     .table("sklad")
    #     .select("id_perfum")
    #     .where("nazwa_skladnika = 'absolut jasminu'")
    #     .orderBy("id_perfum", ascending=False)
    # )
    # sklad_absolut.show(5)
    # sklad_absolut.collect()
    # print( sklad_absolut)
    # perfumy_absolut = perfumy.join(sklad_absolut, on='id_perfum', how='inner')
    # perfumy_absolut.show()
    # ###################
    # data = (
    #     spark.read.format("csv")
    #     .option("inferSchema", "true")
    #     .option("header", "true")
    #     .load("Wyniki.txt")
    # )
    # # df = spark.read.csv("Wyniki.txt")
    # ###################
    # data.printSchema()
    # #####################
    # data2.createOrReplaceTempView("data")
    # spark.sql(
    #     """
    #         SELECT *
    #         FROM data
    #         WHERE Punkty > 17
    #         """
    # ).show()
    # #####################
    # df = spark.read.format("csv").option("header", "true").load("orders/2019.csv")
    # #####################
    # orderSchema = StructType(
    #     [
    #         StructField("SalesOrderNumber", StringType()),
    #         StructField("SalesOrderLineNumber", IntegerType()),
    #         StructField("OrderDate", DateType()),
    #         StructField("CustomerName", StringType()),
    #         StructField("Email", StringType()),
    #         StructField("Item", StringType()),
    #         StructField("Quantity", IntegerType()),
    #         StructField("UnitPrice", FloatType()),
    #         StructField("Tax", FloatType()),
    #     ]
    # )
    # #####################
    # dfx = spark.read.load(
    #     "/home/radek/git_projekty/sparkTraining/datafabrictraining/Files/partitioned_data",
    #     format="parquet",
    # )
    # #####################
    # ## Create Year and Month columns
    # transformed_df = dfall.withColumn("Year", year(col("OrderDate"))).withColumn(
    #     "Month", month(col("OrderDate"))
    # )

    # # Create the new FirstName and LastName fields
    # transformed_df = transformed_df.withColumn(
    #     "FirstName", split(col("CustomerName"), " ").getItem(0)
    # ).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

    # # Filter and reorder columns
    # transformed_df = transformed_df[
    #     "SalesOrderNumber",
    #     "SalesOrderLineNumber",
    #     "OrderDate",
    #     "Year",
    #     "Month",
    #     "FirstName",
    #     "LastName",
    #     "Email",
    #     "Item",
    #     "Quantity",
    #     "UnitPrice",
    #     "Tax",
    # ]

    # # Display the first five orders
    # display(transformed_df.limit(5))
    # transformed_df.show(5)
    # #####################
    pass

Save the transformed data
Add a new cell with the following code to save the transformed dataframe in Parquet format (Overwriting the data if it already exists):

# Save data in partitioned files
# Add a new cell with the following code; which saves the dataframe, partitioning the data by Year and Month:

````
orders_df.write.partitionBy("Year","Month").mode("overwrite").parquet("Files/partitioned_data")
print ("Transf

In [35]:
# %load_ext sparksql_magic
# %config SparkSql.max_num_rows=20

In [47]:
# Assuming you have a DataFrame called 'df'
dfall.createOrReplaceTempView("my_temp_view")
transformed_df.createOrReplaceTempView("my_temp_view2")

In [48]:
# Now you can query the view using Spark SQL
spark.sql("SELECT * FROM my_temp_view").show(5)

+----------------+--------------------+----------+--------------+--------------------+--------------------+--------+---------+--------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|  CustomerName|               Email|                Item|Quantity|UnitPrice|     Tax|
+----------------+--------------------+----------+--------------+--------------------+--------------------+--------+---------+--------+
|         SO49171|                   1|2021-01-01| Mariah Foster|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625| 174.525|
|         SO49172|                   1|2021-01-01|  Brian Howard|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35| 195.468|
|         SO49173|                   1|2021-01-01| Linda Alvarez|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49174|                   1|2021-01-01|Gina Hernandez|gina4@adventure-w...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49178|                   1|2021-01-0

In [ ]:
%%sparksql
SELECT * FROM my_temp_view LIMIT 5

SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,Email,Item,Quantity,UnitPrice,Tax
SO49171,1,2021-01-01,Mariah Foster,mariah21@adventure-works.com,"Road-250 Black, 48",1,2181.5625,174.52499389648438
SO49172,1,2021-01-01,Brian Howard,brian23@adventure-works.com,"Road-250 Red, 44",1,2443.35009765625,195.46800231933594
SO49173,1,2021-01-01,Linda Alvarez,linda19@adventure-works.com,"Mountain-200 Silver, 38",1,2071.419677734375,165.71359252929688
SO49174,1,2021-01-01,Gina Hernandez,gina4@adventure-works.com,"Mountain-200 Silver, 42",1,2071.419677734375,165.71359252929688
SO49178,1,2021-01-01,Beth Ruiz,beth4@adventure-works.com,"Road-550-W Yellow, 44",1,1000.4375,80.03500366210938


In [ ]:
# sqlQuery = "SELECT CAST(YEAR(OrderDate) AS CHAR(4)) AS OrderYear, \
#                 SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue \
#             FROM my_temp_view2 \
#             GROUP BY CAST(YEAR(OrderDate) AS CHAR(4)) \
#             ORDER BY OrderYear"
# df_spark = spark.sql(sqlQuery)
# df_spark.show()

24/03/26 09:07:27 WARN CharVarcharUtils: The Spark cast operator does not support char/varchar type and simply treats them as string type. Please use string type directly to avoid confusion. Otherwise, you can set spark.sql.legacy.charVarcharAsString to true, so that Spark treat them as string type as same as Spark 3.0 and earlier
24/03/26 09:07:27 WARN CharVarcharUtils: The Spark cast operator does not support char/varchar type and simply treats them as string type. Please use string type directly to avoid confusion. Otherwise, you can set spark.sql.legacy.charVarcharAsString to true, so that Spark treat them as string type as same as Spark 3.0 and earlier


+---------+--------------------+
|OrderYear|        GrossRevenue|
+---------+--------------------+
|     2019|   4172169.969970703|
|     2020|   6882259.268127441|
|     2021|1.1547835291696548E7|
+---------+--------------------+



In [ ]:
# dfa = spark.read.format("csv").option("header", "true").load("products/products.csv")

In [ ]:
# display(dfa)
# dfa.show(5)

DataFrame[ProductID: string, ProductName: string, Category: string, ListPrice: string]

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
+---------+--------------------+--------------+---------+
only showing top 5 rows



In [ ]:
# dfa.write.format("delta").saveAsTable("dfa")
# dfa.write.format("delta").saveAsTable("dfa",path="products")
# dfa.write.format("delta").saveAsTable("managed_products")
# dfa.createOrReplaceTempView("dfa")

In [ ]:
# %%sparksql
# DESCRIBE FORMATTED dfa

col_name,data_type,comment
ProductID,string,null
ProductName,string,null
Category,string,null
ListPrice,string,null


In [57]:
# spark.stop()

In [47]:
# %%sparksql
# UPDATE dfa
# SET  ListPrice = ListPrice * 0.9
# WHERE Category = 'Mountain Bikes';

In [48]:
# %reload_ext Kqlmagic

In [74]:
dfdimCustomer_silver.createOrReplaceTempView("dfdimCustomer_silver")

In [75]:
%%sparksql
select * from dfdimCustomer_silver limit 5;


24/03/26 09:15:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Christy Zhu, christy12@adventure-works.com
 Schema: CustomerName, Email
Expected: CustomerName but found: Christy Zhu
CSV file: file:///home/radek/git_projekty/sparkTraining/2019.csv


CustomerName,Email,First,Last
Jonathon Gutierrez,jonathon8@adventure-works.com,Jonathon,Gutierrez
Blake Butler,blake62@adventure-works.com,Blake,Butler
Melissa Perry,melissa2@adventure-works.com,Melissa,Perry
Hailey James,hailey17@adventure-works.com,Hailey,James
Jasmine West,jasmine37@adventure-works.com,Jasmine,West


In [77]:
# %pyspark
df7 = spark.read.load("products.csv", format="csv", header=True)
display(df7.limit(5))

DataFrame[ProductID: string, ProductName: string, Category: string, ListPrice: string]

In [101]:
df8.createOrReplaceTempView("products")

In [102]:
%%sparksql
select * from products limit 10;

ProdID,ProdName,Category,ListPrice
null,ProductName,Category,null
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.989990234375
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.989990234375
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.989990234375
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.989990234375
775,"Mountain-100 Black, 38",Mountain Bikes,3374.989990234375
776,"Mountain-100 Black, 42",Mountain Bikes,3374.989990234375
777,"Mountain-100 Black, 44",Mountain Bikes,3374.989990234375
778,"Mountain-100 Black, 48",Mountain Bikes,3374.989990234375
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.989990234375
